In [2]:
import os
import sys

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [3]:
from core.backtesting.optimizer import BacktestingConfig, BaseStrategyConfigGenerator
from hummingbot.strategy_v2.executors.position_executor.data_types import TrailingStop
from controllers.directional_trading.macd_bb_v1 import MACDBBV1ControllerConfig
from decimal import Decimal


class MACDBBConfigGenerator(BaseStrategyConfigGenerator):
    """
    Strategy configuration generator for MACD and Bollinger Bands optimization.
    """
    async def generate_config(self, trial) -> BacktestingConfig:
        # Suggest hyperparameters using the trial object
        fast_ma = trial.suggest_int("fast_ma", 9, 59, step=10)
        slow_ma = trial.suggest_int("slow_ma", 21, 201, step=10)
        signal_ma = trial.suggest_int("signal_ma", 10, 60, step=10)
        bb_length = trial.suggest_int("bb_length", 10, 200, step=10)
        bb_std = trial.suggest_float("bb_std", 0.5, 2.5, step=0.5)
        total_amount_quote = 1000
        max_executors_per_side = trial.suggest_int("max_executors_per_side", 1, 3)
        take_profit = trial.suggest_float("take_profit", 0.04, 0.05, step=0.01)
        stop_loss = trial.suggest_float("stop_loss", 0.01, 0.05, step=0.01)
        trailing_stop_activation_price = trial.suggest_float("trailing_stop_activation_price", 0.005, 0.02, step=0.005)
        trailing_delta_ratio = trial.suggest_float("trailing_delta_ratio", 0.1, 0.3, step=0.1)
        trailing_stop_trailing_delta = trailing_stop_activation_price * trailing_delta_ratio
        time_limit = 60 * 60 * 24 * 2
        cooldown_time = 60 * 15

        # Create the strategy configuration
        config = MACDBBV1ControllerConfig(
            connector_name="binance_perpetual",
            trading_pair="1000PEPE-USDT",
            macd_fast=fast_ma,
            macd_slow=slow_ma,
            macd_signal=signal_ma,
            bb_length=bb_length,
            bb_std=bb_std,
            total_amount_quote=Decimal(total_amount_quote),
            take_profit=Decimal(take_profit),
            stop_loss=Decimal(stop_loss),
            trailing_stop=TrailingStop(
                activation_price=Decimal(trailing_stop_activation_price),
                trailing_delta=Decimal(trailing_stop_trailing_delta),
            ),
            time_limit=time_limit,
            max_executors_per_side=max_executors_per_side,
            cooldown_time=cooldown_time,
        )

        # Return the configuration encapsulated in BacktestingConfig
        return BacktestingConfig(config=config, start=self.start, end=self.end)

In [4]:
from core.backtesting.optimizer import StrategyOptimizer
import datetime

start_date = datetime.datetime(2024, 8, 1)
end_date = datetime.datetime(2024, 8, 31)
config_generator = MACDBBConfigGenerator(start_date=start_date, end_date=end_date)

optimizer = StrategyOptimizer(root_path=root_path)
await optimizer.optimize(
    study_name="macd_bb_v1_with_optimizer",
    config_generator=config_generator,
    n_trials=100,
)

[I 2024-11-08 14:15:41,164] Using an existing study with name 'macd_bb_v1_with_optimizer' instead of creating a new one.
2024-11-08 14:15:42,414 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fcb2d604fa0>
2024-11-08 14:15:42,415 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7fcb592fb580>, 494849.221547534)]']
connector: <aiohttp.connector.TCPConnector object at 0x7fcb2d6066e0>
2024-11-08 14:15:52,308 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fcb2d6061a0>
2024-11-08 14:15:52,309 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7fcb2d755480>, 494859.11371856)]']
connector: <aiohttp.connector.TCPConnector object at 0x7fcb2d607f10>
2024-11-08 14:15:55,985 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fcb

CancelledError: 

In [5]:
from core.backtesting.optimizer import StrategyOptimizer
import datetime
import asyncio
import aiohttp

# Create an async context manager for the optimizer
class AsyncOptimizer:
    def __init__(self, root_path):
        self.optimizer = StrategyOptimizer(root_path=root_path)
        
    async def __aenter__(self):
        return self.optimizer
        
    async def __aexit__(self, exc_type, exc_val, exc_tb):
        # Clean up any remaining sessions
        await asyncio.sleep(0)  # Allow any pending tasks to complete

async def run_optimization():
    start_date = datetime.datetime(2024, 8, 1)
    end_date = datetime.datetime(2024, 8, 31)
    config_generator = MACDBBConfigGenerator(start_date=start_date, end_date=end_date)
    
    async with AsyncOptimizer(root_path=root_path) as optimizer:
        await optimizer.optimize(
            study_name="macd_bb_v1_with_optimizer",
            config_generator=config_generator,
            n_trials=100,
        )

# Run the optimization
await run_optimization()

[I 2024-11-08 14:17:36,584] Using an existing study with name 'macd_bb_v1_with_optimizer' instead of creating a new one.
2024-11-08 14:17:37,514 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fcaf2721d50>
2024-11-08 14:17:37,515 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7fcaf2a2bd60>, 494959.541359757)]']
connector: <aiohttp.connector.TCPConnector object at 0x7fcaf2722bf0>
2024-11-08 14:17:47,576 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fcaf2721b10>
2024-11-08 14:17:47,577 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7fcaf28a5480>, 494969.6014761)]']
connector: <aiohttp.connector.TCPConnector object at 0x7fcaf2721960>
2024-11-08 14:17:51,206 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fcaf

An error occurred during optimization: 'MACDh_29_21_10'


2024-11-08 14:18:00,119 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fcaf2854b20>
2024-11-08 14:18:00,121 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7fcaf28a4820>, 494982.144441862)]']
connector: <aiohttp.connector.TCPConnector object at 0x7fcaf2854eb0>
2024-11-08 14:18:04,958 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fcaf2722b60>
2024-11-08 14:18:04,963 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7fcaf2826080>, 494985.776515621)]']
connector: <aiohttp.connector.TCPConnector object at 0x7fcaf2722890>
/home/clint/projects/quants-lab/controllers/directional_trading/macd_bb_v1.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

An error occurred during optimization: 'MACDh_29_21_10'


2024-11-08 14:37:45,687 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fcb341260e0>
2024-11-08 14:37:45,691 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7fcaf28254e0>, 496120.827328547)]']
connector: <aiohttp.connector.TCPConnector object at 0x7fcb341277c0>
2024-11-08 14:37:50,640 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fcaf2ac8850>
2024-11-08 14:37:50,642 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7fcaf28a51e0>, 496124.522146148)]']
connector: <aiohttp.connector.TCPConnector object at 0x7fcaf2720fa0>
/home/clint/projects/quants-lab/controllers/directional_trading/macd_bb_v1.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [6]:

from core.backtesting.optimizer import StrategyOptimizer

optimizer = StrategyOptimizer(root_path=root_path)
optimizer.launch_optuna_dashboard()

2024-11-08 14:55:29,637 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fcaf2a7ceb0>
2024-11-08 14:55:29,638 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fcaf2acb2e0>
2024-11-08 14:55:29,639 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fcaf2855d20>


Listening on http://127.0.0.1:8080/
Hit Ctrl-C to quit.

127.0.0.1 - - [08/Nov/2024 14:55:37] "GET / HTTP/1.1" 302 0
127.0.0.1 - - [08/Nov/2024 14:55:37] "GET /dashboard HTTP/1.1" 200 4145
127.0.0.1 - - [08/Nov/2024 14:55:37] "GET /static/bundle.js HTTP/1.1" 200 4158971
127.0.0.1 - - [08/Nov/2024 14:55:38] "GET /api/studies HTTP/1.1" 200 143
127.0.0.1 - - [08/Nov/2024 14:55:38] "GET /favicon.ico HTTP/1.1" 200 7670
127.0.0.1 - - [08/Nov/2024 14:55:43] "GET /api/meta HTTP/1.1" 200 60
/home/clint/miniconda3/envs/quants-lab/lib/python3.10/site-packages/optuna_dashboard/_importance.py:75: ExperimentalWarning: PedAnovaImportanceEvaluator is experimental (supported from v3.6.0). The interface can change in the future.
  return get_param_importances(study, target=target, evaluator=PedAnovaImportanceEvaluator())
127.0.0.1 - - [08/Nov/2024 14:55:43] "GET /api/studies/1?after=0 HTTP/1.1" 200 515234
/home/clint/miniconda3/envs/quants-lab/lib/python3.10/site-packages/optuna/importance/_ped_anova/ev